<a href="https://colab.research.google.com/github/JurijJusel/Image/blob/t6-dir-reader/apple_banana_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from operator import itemgetter
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files
from IPython.display import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
apple_banana_train_path = '/content/drive/MyDrive/Colab Notebooks/dataset/train'

In [6]:
image_url = []
folder_name=[]
for file_path, folders, files in os.walk(apple_banana_train_path):
  for file in files:
      full_path = os.path.join(file_path, file)
      image_url.append(full_path)
      direct = file_path.split('/')[7:]
      folder_name.append('/'.join(direct))


In [7]:
folder_name_df = pd.DataFrame(folder_name, columns=['name'])
img_path_df = pd.DataFrame(image_url,columns=['image_path'] )
df = pd.concat([folder_name_df, img_path_df], axis=1)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/apple_banana.csv', index=False)

In [8]:
apple_banana_df = '/content/drive/MyDrive/Colab Notebooks/apple_banana.csv'

In [10]:
read_apple_banana_df = pd.read_csv(apple_banana_df)
read_apple_banana_df.info()
# read_apple_banana_df.describe()
read_apple_banana_df.head()
# read_apple_banana_df.apply(lambda x: x.upper())
# read_apple_banana_df.tail(3)  # paskutiniai 3 eilutes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        143 non-null    object
 1   image_path  143 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


,name,image_path
0,apple,/content/drive/MyDrive/Colab Notebooks/dataset...
1,apple,/content/drive/MyDrive/Colab Notebooks/dataset...
2,apple,/content/drive/MyDrive/Colab Notebooks/dataset...
3,apple,/content/drive/MyDrive/Colab Notebooks/dataset...
4,apple,/content/drive/MyDrive/Colab Notebooks/dataset...


In [38]:
read_apple_banana_df['image_path']


0      /content/drive/MyDrive/Colab Notebooks/dataset...
1      /content/drive/MyDrive/Colab Notebooks/dataset...
2      /content/drive/MyDrive/Colab Notebooks/dataset...
3      /content/drive/MyDrive/Colab Notebooks/dataset...
4      /content/drive/MyDrive/Colab Notebooks/dataset...
                             ...                        
138    /content/drive/MyDrive/Colab Notebooks/dataset...
139    /content/drive/MyDrive/Colab Notebooks/dataset...
140    /content/drive/MyDrive/Colab Notebooks/dataset...
141    /content/drive/MyDrive/Colab Notebooks/dataset...
142    /content/drive/MyDrive/Colab Notebooks/dataset...
Name: image_path, Length: 143, dtype: object

In [ ]:
for line in range(len(read_apple_banana_df)):
    line_path=read_apple_banana_df['image_path']


In [ ]:
image_lst=[]
folder_name=[]
for line in line_path:
    # print(line)
    img_path = cv2.imread(line)
    direct = line.split('/')[7:8]
    folder_name.append('/'.join(direct))
    img = plt.imread(line)
    gray_img = cv2.cvtColor(img_path, cv2.COLOR_RGB2GRAY)
    img_resized = cv2.resize(gray_img, (122,122))
    img_array = np.array(img_resized)
    image_gray_lst=[item for sublist in img_array for item in sublist]
    image_lst.append(image_gray_lst)


In [ ]:
img_data_df = pd.DataFrame(image_lst)
name_df = pd.DataFrame(folder_name, columns=['image_name'])
apple_banana_df_grey = pd.concat([img_data_df, name_df], axis=1)
apple_banana_df_grey.to_csv('/content/drive/MyDrive/Colab Notebooks/apple_banana_grey_df.csv', index=False)
       

In [22]:
apple_banana_grey_df = '/content/drive/MyDrive/Colab Notebooks/apple_banana_grey_df.csv'
read_df = pd.read_csv(apple_banana_grey_df)
# read_df.info()
# read_df.describe()
# read_df.head()
read_df
# read_df.tail(3)  # paskutiniai 3 eilutes

,0,1,2,3,4,5,6,7,8,9,...,14875,14876,14877,14878,14879,14880,14881,14882,14883,image_name
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,apple
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,apple
2,-122,-121,-123,126,-126,-120,-118,-117,-118,-121,...,-18,-29,-37,-42,-43,-33,-51,-41,-44,apple
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,apple
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,apple
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,banana
139,-76,-68,-61,-42,-32,-43,-54,-53,-79,-82,...,-102,-2,-1,-1,-1,-1,-1,-1,-1,banana
140,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,banana
141,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,banana


In [11]:
# from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

In [23]:
read_df['image_name']

0       apple
1       apple
2       apple
3       apple
4       apple
        ...  
138    banana
139    banana
140    banana
141    banana
142    banana
Name: image_name, Length: 143, dtype: object

In [27]:
#making instance of labelencoder()
le = LabelEncoder()
encoded = le.fit_transform(read_df['image_name'])
print(encoded)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [30]:
# removing the original column 'Purchased' from df
read_df.drop('image_name', axis=1, inplace=True)
 
# Appending the array to our dataFrame
read_df['image_name'] = encoded
# printing Dataframe
read_df

,0,1,2,3,4,5,6,7,8,9,...,14875,14876,14877,14878,14879,14880,14881,14882,14883,image_name
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
2,-122,-121,-123,126,-126,-120,-118,-117,-118,-121,...,-18,-29,-37,-42,-43,-33,-51,-41,-44,0
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
139,-76,-68,-61,-42,-32,-43,-54,-53,-79,-82,...,-102,-2,-1,-1,-1,-1,-1,-1,-1,1
140,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,-2,-2,-2,1
141,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [31]:
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(read_df,read_df, test_size=0.33, random_state=42)

In [35]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,14875,14876,14877,14878,14879,14880,14881,14882,14883,image_name
25,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
23,8,8,8,9,9,9,9,9,9,9,...,123,-121,-115,-106,-112,-104,-107,-110,-115,0
115,33,34,36,35,39,45,51,48,39,38,...,77,75,75,74,73,72,71,68,68,1
67,-6,-5,-6,-5,-4,-4,-6,-66,59,-45,...,16,12,17,6,45,50,40,24,16,0
44,35,35,35,35,35,35,35,35,35,35,...,-68,-69,-69,-68,-70,-70,-70,-70,-70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
92,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [36]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,14875,14876,14877,14878,14879,14880,14881,14882,14883,image_name
117,110,101,78,41,116,48,57,72,62,21,...,76,72,77,73,71,72,67,77,74,1
19,112,116,118,122,120,-126,126,124,126,120,...,90,92,91,73,80,77,76,77,76,0
82,50,49,45,33,26,33,30,27,34,38,...,-119,-77,-68,-69,98,126,16,126,31,1
97,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,1
56,45,37,45,117,-76,-60,-122,72,45,40,...,57,56,55,66,62,68,71,79,86,0
12,-14,-13,-13,-13,-14,-13,-13,-13,-13,-12,...,-9,-11,-10,-10,-8,-8,-8,-7,-8,0
131,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
65,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
66,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-101,-101,-101,-101,-101,-101,-101,-101,-101,0
18,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0


In [37]:
y_train

,0,1,2,3,4,5,6,7,8,9,...,14875,14876,14877,14878,14879,14880,14881,14882,14883,image_name
25,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
23,8,8,8,9,9,9,9,9,9,9,...,123,-121,-115,-106,-112,-104,-107,-110,-115,0
115,33,34,36,35,39,45,51,48,39,38,...,77,75,75,74,73,72,71,68,68,1
67,-6,-5,-6,-5,-4,-4,-6,-66,59,-45,...,16,12,17,6,45,50,40,24,16,0
44,35,35,35,35,35,35,35,35,35,35,...,-68,-69,-69,-68,-70,-70,-70,-70,-70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
92,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [38]:
y_test

,0,1,2,3,4,5,6,7,8,9,...,14875,14876,14877,14878,14879,14880,14881,14882,14883,image_name
117,110,101,78,41,116,48,57,72,62,21,...,76,72,77,73,71,72,67,77,74,1
19,112,116,118,122,120,-126,126,124,126,120,...,90,92,91,73,80,77,76,77,76,0
82,50,49,45,33,26,33,30,27,34,38,...,-119,-77,-68,-69,98,126,16,126,31,1
97,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,1
56,45,37,45,117,-76,-60,-122,72,45,40,...,57,56,55,66,62,68,71,79,86,0
12,-14,-13,-13,-13,-14,-13,-13,-13,-13,-12,...,-9,-11,-10,-10,-8,-8,-8,-7,-8,0
131,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
65,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
66,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-101,-101,-101,-101,-101,-101,-101,-101,-101,0
18,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
